In [1]:
import pandas as pd

In [18]:
from datetime import datetime,timedelta

## Reading The CSV

In [2]:
df = pd.read_csv('/home/user/Downloads/GOOG.csv')

In [3]:
df.head()

,Date,Close Price,Open Price,High Price,Low Price,Total Traded Quantity
0,"Oct 24, 2019","1,259.11","1,259.11","1,262.90","1,252.35",1.01M
1,"Oct 23, 2019","1,257.63","1,240.21","1,258.04","1,240.21",1.18M
2,"Oct 22, 2019","1,241.20","1,244.48","1,248.73","1,239.85",1.28M
3,"Oct 21, 2019","1,244.28","1,248.70","1,253.51","1,239.99",1.03M
4,"Oct 18, 2019","1,244.41","1,254.69","1,258.11","1,240.14",1.58M


In [4]:
df.isna().sum()

Date                     0
Close Price              0
Open Price               0
High Price               0
Low Price                0
Total Traded Quantity    0
dtype: int64

In [5]:
df.shape

(503, 6)

In [6]:
df.describe()

,Date,Close Price,Open Price,High Price,Low Price,Total Traded Quantity
count,503,503,503,503,503,503
unique,503,496,494,492,492,229
top,"Mar 19, 2019","1,145.34","1,172.00","1,088.00","1,042.00",1.77M
freq,1,2,2,3,2,10


In [7]:
df.columns

Index(['Date', 'Close Price', 'Open Price', 'High Price', 'Low Price',
       'Total Traded Quantity'],
      dtype='object')

## Cleaning Prices and Converting to Float

In [8]:
df2 = df.copy()

In [9]:
for i in ['Close Price', 'Open Price', 'High Price', 'Low Price']:
    df2[i] = df2[i].map(lambda x: ''.join(x.split(',')))

In [10]:
df2['Total Traded Quantity'] = df2['Total Traded Quantity'].map(lambda x: (x.split('M')[0]) if 'M' in x else x)

In [11]:
df2['Total Traded Quantity'] = df2['Total Traded Quantity'].map(lambda x: 0.001*float(x.split('K')[0]) if 'K' in x else x)

In [12]:
for i in ['Close Price', 'Open Price', 'High Price', 'Low Price','Total Traded Quantity']:
    df2[i] = df2[i].astype('float32')

In [13]:
df2.dtypes

Date                      object
Close Price              float32
Open Price               float32
High Price               float32
Low Price                float32
Total Traded Quantity    float32
dtype: object

## Analyzing Prices

In [14]:
df2.describe()

,Close Price,Open Price,High Price,Low Price,Total Traded Quantity
count,503.000000,503.000000,503.000000,503.000000,503.000000
mean,1132.038818,1131.834473,1142.713989,1120.766113,1.795025
std,70.086967,69.720551,68.474480,71.023315,0.850383
min,984.669983,984.320007,994.429993,977.659973,0.668240
25%,1073.645020,1073.960022,1084.984985,1062.375000,1.270000
50%,1128.630005,1130.000000,1139.300049,1116.790039,1.580000
75%,1189.914978,1188.000000,1199.089966,1174.924988,2.030000
max,1296.199951,1289.119995,1296.969971,1271.709961,6.660000


## Converting Dates to Datetime

In [15]:
df2.Date = pd.to_datetime(df2.Date)

In [19]:
prev = df2.Date[0]-timedelta(90)

In [20]:
prev

Timestamp('2019-07-26 00:00:00')

In [21]:
dat = df2[df2.Date>=prev]

In [22]:
print('Last 90 Days:\nMax Price = ',dat['Close Price'].max(),'\nMin Price = ',dat['Close Price'].min(),'\nMean Price = ',dat['Close Price'].mean())

Last 90 Days:
Max Price =  1259.11 
Min Price =  1153.58 
Mean Price =  1209.1167


## Subtracting Minimum Date From Maximum Date:

In [23]:
df2.Date.iloc[-1] - df2.Date[0]

Timedelta('-729 days +00:00:00')

In [24]:
df2['Month'] = df2.Date.map(lambda x: x.strftime('%b'))

In [25]:
df2['Year'] = df2.Date.map(lambda x: x.strftime('%Y'))

In [26]:
df2['temp'] = df2['Month']+'-'+df2['Year']

## Calculating Monthly VWAPs

In [27]:
vwap = pd.DataFrame()

In [28]:
def calc(df):
    p = df['Close Price'].values
    v = df['Total Traded Quantity'].values
    return df.assign(vwap=(p*v).cumsum()/v.cumsum())

In [29]:
df3 = df2.groupby(('Month','Year'),group_keys=False).apply(calc)

/home/user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


In [32]:
val = df3.temp

In [43]:
vwap['Date'] = val.unique()

In [49]:
li = []

In [50]:
for i in vwap.Date:
    li.append((df3[df3['temp']==i]['vwap'].sum())/len((df3[df3['temp']==i]['vwap'])))

In [51]:
vwap['vwap'] = li

In [52]:
vwap

,Date,vwap
0,Apr-2018,1039.895740
1,Apr-2019,1238.399554
2,Aug-2018,1241.184528
3,Aug-2019,1180.048118
4,Dec-2017,1061.661523
5,Dec-2018,1034.177118
6,Feb-2018,1100.108655
7,Feb-2019,1121.357011
8,Jan-2018,1167.442522
9,Jan-2019,1094.331752


## Last n day's Average Price

In [53]:
def avg(n):
    prev = df2.Date[0]-timedelta(n)
    dat = df2[df2['Date']>=prev]
    print('Average Price for the last ',n,' days = ',dat['Close Price'].sum()/len(dat))

In [54]:
avg(7)
avg(14)
avg(30)
avg(90)
avg(180)
avg(360)

Average Price for the last  7  days =  1249.9049479166667
Average Price for the last  14  days =  1238.8744673295455
Average Price for the last  30  days =  1224.624320652174
Average Price for the last  90  days =  1209.11669921875
Average Price for the last  180  days =  1169.7704613095239
Average Price for the last  360  days =  1145.0316265060242


In [55]:
def chg(n):
    new = df2['Close Price'][0]
    try:
        prev = df2.Date[0]-timedelta(n)
        orig = df2[df2['Date']==prev]['Close Price'].values
        temp = ((new-orig)/orig)[0]*100
    except IndexError:
        prev = df2.Date[0]-timedelta(n+1)
        orig = df2[df2['Date']==prev]['Close Price'].values
        temp = ((new-orig)/orig)[0]*100
    print('Percentage Change over last ',n,' days = ',temp,'%')

In [56]:
chg(7)
chg(14)
chg(30)
chg(90)
chg(180)
chg(360)

Percentage Change over last  7  days =  0.5036666989326477 %
Percentage Change over last  14  days =  4.104278236627579 %
Percentage Change over last  30  days =  3.347207233309746 %
Percentage Change over last  90  days =  1.1154667474329472 %
Percentage Change over last  180  days =  -1.433362439274788 %
Percentage Change over last  360  days =  21.684885025024414 %


In [77]:
df3['Day_Perc_Change'] = 0

In [78]:
df3['Day_Perc_Change'] = df3['Close Price'].pct_change()*100

In [79]:
df3['Day_Perc_Change'].iloc[0] = 0

/home/user/.local/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [123]:
df3['Trend'] = ''

In [124]:
df3.loc[(df3['Day_Perc_Change']>-0.5)&(df3['Day_Perc_Change']<0.5),'Trend'] = 'Slight or No Change'
df3.loc[(df3['Day_Perc_Change']>=0.5)&(df3['Day_Perc_Change']<1),'Trend'] = 'Slight Positive'
df3.loc[(df3['Day_Perc_Change']>-1)&(df3['Day_Perc_Change']<=-0.5),'Trend'] = 'Slight Negative'
df3.loc[(df3['Day_Perc_Change']>=1)&(df3['Day_Perc_Change']<3),'Trend'] = 'Positive'
df3.loc[(df3['Day_Perc_Change']>-3)&(df3['Day_Perc_Change']<=-1),'Trend'] = 'Negative'
df3.loc[(df3['Day_Perc_Change']>=3)&(df3['Day_Perc_Change']<7),'Trend'] = 'Among Top Gainers'
df3.loc[(df3['Day_Perc_Change']>-7)&(df3['Day_Perc_Change']<=-3),'Trend'] = 'Among Top Losers'
df3.loc[(df3['Day_Perc_Change']>=7),'Trend'] = 'Bull Run'
df3.loc[(df3['Day_Perc_Change']<=-7),'Trend'] = 'Bear Drop'

## Median

In [130]:
df3.groupby('Trend')['Total Traded Quantity'].median()

Trend
Among Top Gainers      2.305
Among Top Losers       1.920
Bear Drop              2.250
Bull Run               3.550
Negative               1.500
Positive               1.665
Slight Negative        1.490
Slight Positive        1.450
Slight or No Change    1.580
Name: Total Traded Quantity, dtype: float32

## Average

In [131]:
df3.groupby('Trend')['Total Traded Quantity'].mean()

Trend
Among Top Gainers      2.577178
Among Top Losers       2.204632
Bear Drop              2.056667
Bull Run               3.468000
Negative               1.726792
Positive               1.783822
Slight Negative        1.712321
Slight Positive        1.720406
Slight or No Change    1.689781
Name: Total Traded Quantity, dtype: float32

In [133]:
df3.to_csv('Module1.csv')

In [134]:
df3

,Date,Close Price,Open Price,High Price,Low Price,Total Traded Quantity,Month,Year,temp,vwap,Day_Perc_Change,Trend
375,2018-04-30,1018.580017,1034.420044,1038.369995,1018.299988,1.72000,Apr,2018,Apr-2018,1018.580017,0.000000,Slight or No Change
376,2018-04-27,1031.449951,1045.540039,1051.689941,1027.589966,2.04000,Apr,2018,Apr-2018,1025.562744,1.263523,Positive
377,2018-04-26,1043.310059,1033.219971,1052.020020,1020.450012,2.55000,Apr,2018,Apr-2018,1032.734863,1.149845,Positive
378,2018-04-25,1022.989990,1029.750000,1036.069946,1017.010010,2.89000,Apr,2018,Apr-2018,1029.673706,-1.947653,Negative
379,2018-04-24,1022.640015,1059.199951,1064.489990,1012.859985,6.41000,Apr,2018,Apr-2018,1026.785522,-0.034213,Slight or No Change
...,...,...,...,...,...,...,...,...,...,...,...,...
33,2019-09-09,1205.270020,1207.079956,1220.550049,1193.170044,1.59000,Sep,2019,Sep-2019,1227.598267,-0.035655,Slight or No Change
34,2019-09-06,1206.319946,1209.140015,1212.939941,1203.260010,0.94858,Sep,2019,Sep-2019,1226.705078,0.087106,Slight or No Change
35,2019-09-05,1212.189941,1193.660034,1214.000000,1192.689941,1.32000,Sep,2019,Sep-2019,1225.904053,0.486600,Slight or No Change
36,2019-09-04,1182.270020,1179.449951,1184.880005,1171.530029,1.07000,Sep,2019,Sep-2019,1224.035767,-2.468252,Negative
